In [ ]:
#default_exp train3d

In [ ]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

In [ ]:
torch.cuda.set_device(2)

In [ ]:
df_any.head()

In [ ]:
df_any.groupby(['SeriesInstanceUID']).agg(['count'])

In [6]:
#export
class OpenCTs:
    def __init__(self, path): 
        self.fn = get_pil_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [7]:
#export
class TfmSOP:
    def __init__(self,df,open_fn):
        self.open_fn = open_fn
        self.df = df
    
    def x(self, sid):
        sids = self.df.SOPInstanceUID[sid].values
        return self.open_fn(sids)
    
    def y(self, sid): return TensorMultiCategory(self.df.loc[sid,htypes].values).float()

In [8]:
#export
def get_3d_dsrc(df, open_fn):
    df_series = df.reset_index().set_index('SeriesInstanceUID').sort_values("ImagePositionPatient2")
    tfm = TfmSOP(df_series, open_fn)
    sids = df_series.index.unique()
    splits = RandomSplitter(valid_pct=0.1, seed=42)(sids)
    dsrc = DataSource(sids, [[tfm.x],[tfm.y]], splits=splits)
    return dsrc

In [9]:

def get_3d_dbunch(df, path=path_jpg256):
    dsrc = get_3d_dsrc(df, open_fn=OpenCTs(path))

    nrm = Normalize.from_stats(mean,std)
    batch_tfms = L(nrm, Cuda(), IntToFloatTensor())

    dbunch = DataBunch(
        TfmdDL(dsrc.train, bs=None, after_batch=batch_tfms, num_workers=8, shuffle=True),
        TfmdDL(dsrc.valid, bs=None, after_batch=batch_tfms, num_workers=8)
    )
    dbunch.device = default_device()
    dbunch.c = 6
    return dbunch


In [ ]:
gr

In [10]:
dbunch = get_3d_dbunch(df_any)

In [11]:
x,y = dbunch.one_batch()
x.shape, y.shape

(torch.Size([34, 3, 256, 256]), torch.Size([34, 6]))

## Features

In [14]:
#export
def get_np_fn(p):
    def _f(fn): return torch.from_numpy(np.load(str(p/f'{fn}.npy')))
    return _f

In [15]:
#export
class OpenFeats:
    def __init__(self, path):
        self.fn = get_np_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [16]:
dsrc_feat = get_3d_dsrc(df_any, open_fn=OpenFeats(path/'features_224'))

In [17]:
dbunch_feat = DataBunch(
    TfmdDL(dsrc.train, bs=None, after_batch=[Cuda()], num_workers=8, shuffle=True),
    TfmdDL(dsrc.valid, bs=None, after_batch=[Cuda()], num_workers=8)
)
dbunch_feat.device = default_device()
dbunch_feat.c = 6

In [18]:
dsrc_feat[0][0].shape

torch.Size([35, 512, 7, 7])

## Model

In [19]:
#export
class Batchify(Module):
    def forward(self, x): return x[None].transpose(1,2)

class DeBatchify(Module):
    def forward(self, x): return x[0].transpose(0,1)

def conv3(ni,nf,stride=1):
    return ConvLayer(ni, nf, (5,3,3), stride=(1,stride,stride), ndim=3, padding=(2,1,1))

In [20]:
#export
def get_3d_head():
    m = nn.Sequential(Batchify(),
        conv3(512,256,2), # 8
        conv3(256,128,2), # 4
        conv3(128, 64,2), # 2
        DeBatchify(), nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(64,6))
    init_cnn(m)
    return m

In [21]:
xb, yb = next(iter(dbunch_feat.train_dl))

In [22]:
xb.shape

torch.Size([32, 3, 256, 256])

In [ ]:
m = get_3d_head()

In [64]:
config=dict(custom_head=m, init=None)
learn = get_learner(dbunch, xresnet34, get_loss(), config=config)

In [14]:
learn.summary()

Sequential (Input shape: 28 x 3 x 256 x 256)
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               28 x 32 x 128 x 128  864        True      
________________________________________________________________
BatchNorm2d          28 x 32 x 128 x 128  64         True      
________________________________________________________________
ReLU                 28 x 32 x 128 x 128  0          False     
________________________________________________________________
Conv2d               28 x 32 x 128 x 128  9,216      True      
________________________________________________________________
BatchNorm2d          28 x 32 x 128 x 128  64         True      
________________________________________________________________
ReLU                 28 x 32 x 128 x 128  0          False     
________________________________________________________________
Conv2d               28 x 64 x 128 x 128  18,432     True      
_____________________________________________________

## Training

In [18]:
name = 'train3d_sample'

In [ ]:
do_fit(learn, 1, 4e-2, freeze=False)
learn.save(f'runs/{name}-1')

## Export

In [12]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data_01_metadata_stage2.ipynb.
Converted 01_data_02_preprocess_windows.ipynb.
Converted 02_train_00_train.ipynb.
Converted 02_train_01_train_full.ipynb.
Converted 02_train_02_train_wgt.ipynb.
Converted 03_train3d_00_train3d.ipynb.
Converted 03_train3d_01_train3d.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 03_train3d_scratch_3Dbatch.ipynb.
Converted 03_train3d_scratch_extract_features.ipynb.
